# Writing SPARQL queries programmatically

This notebook demonstrates how to compose SPARQL queries programmatically, with little to no prior knowledge of SPARQL syntax.

As an example, we'll work with the well-known Pizza ontology and a set of pre-loaded RDF data.

In [1]:
from tools4rdf.network.network import OntologyNetwork
from rdflib import Graph

In [2]:
onto = OntologyNetwork("pizza.owl")

<img src="pizza_schematic.jpg" alt="Pizza schematic" width="400"/>


In [3]:
g = Graph()
g.parse("pizza_kg.ttl", format="ttl")

<Graph identifier=N6b12024b93dc4c499a0d9a1800c7452f (<class 'rdflib.graph.Graph'>)>

Naturally, you can write and run SPARQL queries directly. Here's a simple example:

In [4]:
query = """
PREFIX pizza: <https://example.org/pizza#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
SELECT DISTINCT ?Food ?hasPricevalue
WHERE {
    ?Food pizza:hasPrice ?hasPricevalue .
}"""

Running a Direct SPARQL Query Using RDFLib

In [5]:
results = g.query(query)

In [6]:
for row in results:
    print(row)

(rdflib.term.URIRef('American:3775bb76'), rdflib.term.Literal('10.0', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#float')))
(rdflib.term.URIRef('AmericanHot:b3f8f95b'), rdflib.term.Literal('11.5', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#float')))
(rdflib.term.URIRef('VegetarianPizza:48687fff'), rdflib.term.Literal('11.5', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#float')))
(rdflib.term.URIRef('IceCream:58e21cdd'), rdflib.term.Literal('7.0', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#float')))
(rdflib.term.URIRef('Margherita:52ceb1ed'), rdflib.term.Literal('8.99', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#float')))
(rdflib.term.URIRef('Margherita:fed8acfc'), rdflib.term.Literal('9.5', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#float')))
(rdflib.term.URIRef('Mushroom:1e428ac1'), rdflib.term.Literal('11.0', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#floa

You can also construct this query programmatically. Here’s how that looks:

You can use `onto.terms` with tab completion to explore available ontology terms.

In [7]:
q = onto.query(g, onto.terms.pizza.Food.any, onto.terms.pizza.hasPrice)
q

,Food,hasPricevalue
0,American:3775bb76,10.0
1,AmericanHot:b3f8f95b,11.5
2,VegetarianPizza:48687fff,11.5
3,IceCream:58e21cdd,7.0
4,Margherita:52ceb1ed,8.99
5,Margherita:fed8acfc,9.5
6,Mushroom:1e428ac1,11.0
7,SpicyPizza:2a9eb8e8,11.0
8,SpicyPizza:8864a1fc,12.99


The `tools4RDF` package also provides a query method that returns results as a pandas DataFrame, making further analysis easier.

Now let's look at how to build this type of query programmatically.
The function requires a source class and one or more destination classes.
In this case, we want to find all pizzas that include a Peperoni Sausage Topping.

In [8]:
q = onto.query(g, onto.terms.pizza.Pizza, onto.terms.pizza.PeperoniSausageTopping)
q

,Pizza,PeperoniSausageTopping


What happened here? The function tried to find the shortest path between the `Pizza` and `PeperoniSausageTopping` classes. Since both are subclasses of `Food`, it initially chose the `hasIngredient` property.

However, by increasing the `num_paths` parameter, we can get alternative paths. In this case, the second option uses `hasTopping`, which is the more appropiate query.

In [9]:
q = onto.query(g, onto.terms.pizza.Pizza, onto.terms.pizza.PeperoniSausageTopping, num_paths=2)
q

,Pizza,PeperoniSausageTopping
0,American:3775bb76,PeperoniSausageTopping:f3a39bb8
1,AmericanHot:b3f8f95b,PeperoniSausageTopping:f3a39bb8


Another interesting aspect here is that although we queried for `Pizza`, we got `American` and `AmericanHot` pizzas, which are subclasses. `tools4RDF` automatically picks a class and its subclasses for the query. You can change this by using `Pizza.only`:

In [10]:
q = onto.query(g, onto.terms.pizza.Pizza.only, onto.terms.pizza.PeperoniSausageTopping, num_paths=2)
q

,Pizza,PeperoniSausageTopping


We do not get any results here, because our knowledge graph does not contain any instance of `Pizza`, only its subclasses.

If you know part of the path in advance, you can specify intermediate classes or properties.
For example, we can directly query for any class that has a `PeperoniSausageTopping` using the `hasTopping` object property.

In [11]:
q = onto.query(g, onto.terms.pizza.Pizza.any, 
               [[onto.terms.pizza.hasTopping, onto.terms.pizza.PeperoniSausageTopping],])
q

,Pizza,hasTopping_PeperoniSausageTopping
0,American:3775bb76,PeperoniSausageTopping:f3a39bb8
1,AmericanHot:b3f8f95b,PeperoniSausageTopping:f3a39bb8


Alternatively, you can also start from a specific predicate and build the query outward from there.

In [12]:
q = onto.query(g, onto.terms.pizza.hasTopping, onto.terms.pizza.PeperoniSausageTopping)
q


,LaReine,hasTopping_PeperoniSausageTopping
0,American:3775bb76,PeperoniSausageTopping:f3a39bb8
1,AmericanHot:b3f8f95b,PeperoniSausageTopping:f3a39bb8
